In [34]:
#   /$$$$$$                        /$$                                              /$$$$$$  /$$$$$$
#  /$$__  $$                      | $$                                             /$$__  $$|_  $$_/
# | $$  \__/  /$$$$$$   /$$$$$$  /$$$$$$   /$$$$$$/$$$$   /$$$$$$  /$$$$$$$       | $$  \ $$  | $$  
# | $$ /$$$$ |____  $$ /$$__  $$|_  $$_/  | $$_  $$_  $$ |____  $$| $$__  $$      | $$$$$$$$  | $$  
# | $$|_  $$  /$$$$$$$| $$  \__/  | $$    | $$ \ $$ \ $$  /$$$$$$$| $$  \ $$      | $$__  $$  | $$  
# | $$  \ $$ /$$__  $$| $$        | $$ /$$| $$ | $$ | $$ /$$__  $$| $$  | $$      | $$  | $$  | $$  
# |  $$$$$$/|  $$$$$$$| $$        |  $$$$/| $$ | $$ | $$|  $$$$$$$| $$  | $$      | $$  | $$ /$$$$$$
#  \______/  \_______/|__/         \___/  |__/ |__/ |__/ \_______/|__/  |__/      |__/  |__/|______/


%matplotlib inline
import pandas as pd
import arrow
from termcolor import colored
from datetime import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

print_queue = []

def encode_vals(col):
    items = list()
    for item in col.values:
        item = item[0]
        if isinstance(item, float): item = "None"
        if "Bear" in item: items.append(-1)
        elif "Bull" in item: items.append(1)
        else: items.append(0)
    return items

def encode_dates(col):
    items = list()
    for item in col.values:
        split = item[0].split("/")
        fill_2 = lambda x: x if len(x)==2 else "0"+str(x)
        item = "{}/{}/{}".format(fill_2(split[0]), fill_2(split[1]), split[2])
        items.append(arrow.get(item, 'MM/DD/YYYY'))
    return items

class Trade(object):
    def __init__(self, enter_date, exit_date, name, share_multiplyer, **params):
        global print_queue
        for key, value in params.items():
            setattr(self, key, value)
        self.MULTIPLYER = {'oil': self.OIL_MULTIPLYER, 'equities': self.EQUITY_MULTIPLIER}
        self.enter_date, self.exit_date = enter_date, exit_date
        self.name = name
        self.share_multiplyer = share_multiplyer*self.MULTIPLYER[name]
        self.entered, self.exitted = False, False
    
    def enter(self, cur_date, money):
        if cur_date == self.enter_date:
            self.entered = True
            formatted_date = cur_date.format('MM/DD/YYYY')
            self.initial_stock_value = data[self.enter_date][matcher[self.name]]
            self.shares = self.share_multiplyer*((money*self.PERCENT_CAPITAL_PER_TRADE)/self.initial_stock_value)
            self.inital_value = self.initial_stock_value * self.shares
            if self.shares > 0: action = colored('Buying', 'green')
            else: action = colored('Shorting', 'red')
            print_queue.append("{}: {} {:.2f} shares of {} for ${:.2f}" \
                  .format(formatted_date, action, abs(self.shares), self.name, abs(self.inital_value)))
            return 0
#             return self.inital_value
        else: return 0

    def exit(self, cur_date, force=False):
        if not self.entered: return 0
        if self.exitted: return 0
        try: exit_stock_value = data[self.exit_date][matcher[self.name]]
        except KeyError: # no data for that day
            try: exit_stock_value = data[self.exit_date.replace(day=1)][matcher[self.name]]
            except KeyError: exit_stock_value = data[self.exit_date.replace(day=2)][matcher[self.name]]
        cur_stop_loss = abs((1-self.STOP_LOSS)*self.initial_stock_value)
        cur_stop_gain = abs((1+self.STOP_GAIN)*self.initial_stock_value)
        if force or exit_stock_value<=cur_stop_loss or exit_stock_value>=cur_stop_gain:
            formatted_date = cur_date.format('MM/DD/YYYY')
            exit_value = exit_stock_value * self.shares
            self.profit = exit_value - self.inital_value
            if self.INVERSE_GARTMAN: self.profit = -self.profit
            if self.shares > 0: action = 'Selling' #action = colored('Selling', 'green')
            else: action = 'Covering' #action = colored('Covering', 'red')
            if self.profit > 0: profit = colored("(profit: ${:.2f})".format(self.profit), 'green')
            else: profit = colored("(loss: ${:.2f})".format(abs(self.profit)), 'red')
            print_queue.append("{}: {} {:.2f} shares of {} for ${:.2f} {}" \
                  .format(formatted_date, action, abs(self.shares), self.name, abs(exit_value), profit))
            self.exitted = True
            return self.profit
        else: return 0

        
def make_trades(trades, trade_kwargs):
    for trade_ in trades.iterrows():
        trade = trade_[1]
        if trade['Stocks'] != 0: name = "equities"; share_multiplyer = trade['Stocks']
        elif trade['Crude'] != 0: name = "oil"; share_multiplyer = trade['Crude']
        else: raise ValueError

        enter_date = trade['Date']
        exit_date = enter_date.replace(months=1)
        yield Trade(enter_date, exit_date, name, share_multiplyer, **trade_kwargs)

df = pd.read_csv("GartmanRecords2016.csv")[['Date', 'Stocks', 'Crude']]
df['Stocks'], df['Crude'] = map(encode_vals, [df[['Stocks']], df[['Crude']]])
df['Date'] = encode_dates(df[['Date']])
trades_table = df

data = pd.read_csv("HistoricalData.csv", names=['Date', 'Equities', 'Oil'])
data.drop(data.index[[0]], inplace=True) # delete header row
data['Equities'], data['Oil'] = pd.to_numeric(data['Equities'].values), pd.to_numeric(data['Oil'].values)
data['Date'] = encode_dates(data[['Date']])
data.set_index('Date', inplace=True)
data = data.T.to_dict("list")

matcher = {"equities": 0, "oil": 1}
        
def main(**kwargs):
    global print_queue
    print_queue = []
    MULTIPLYER = {'oil': kwargs['OIL_MULTIPLYER'], 'equities': kwargs['EQUITY_MULTIPLIER']}
    trades = list(make_trades(trades_table, kwargs))
    money = kwargs['INITIAL_MONEY']
    portfolio = []
    start = datetime(2016, 1, 1, 0, 0) # Jan 1, 2016
    end = datetime(2017, 1, 1, 0, 0)   # Jan 1, 2017
    date_range = arrow.Arrow.range('day', start, end)

    for ind, date in enumerate(date_range):
        enters = [trade.enter(date, money) for trade in trades]
        if ind != len(date_range)-1: profit = sum([trade.exit(date) for trade in trades])
        else: profit = sum([trade.exit(date, force=True) for trade in trades])
        money += profit
        portfolio.append(money)

    fig = plt.figure(figsize=(11, 8.5))
    ax = fig.add_subplot(111)
    ax.plot(portfolio)
    # ax.set_ylim(0, max(portfolio))
    ax.set_title("The Gartman Portfolio")
    ax.set_xlabel("Days after Jan 1, 2016")
    ax.set_ylabel("Gartman's Assets ($)")
    plt.show()
    print "With Gartman's guidance, we went from ${:.2f} to ${:.2f}!".format(kwargs['INITIAL_MONEY'], money)
    print "\n"
    print "\n".join(print_queue)

interact(main, INVERSE_GARTMAN = False, STOP_LOSS = (0.01, 0.25, 0.01), STOP_GAIN = (0.01, 0.25, 0.01), \
         OIL_MULTIPLYER=(0.1, 5, 0.1), EQUITY_MULTIPLIER=(0.1, 5, 0.1), PERCENT_CAPITAL_PER_TRADE = (0.1, 1, 0.01), 
         INITIAL_MONEY = (0, 100000, 100))
print